In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import RegexpTokenizer

import os
import sys

sys.path.append("../../data_preprocess/")

import pickle
import re
from utils import json_pretty_dump, word2VecContinueLearning, trainWord2VecModelType2, tokenizeData, convertWord2Vec, text_cleansing, parse_datetime
seed = 42
np.random.seed(seed)

data_dir = "../processed_type2/AIT_preprocessed_type2"

label_dir = [
    "mail.cup.com",
    "mail.insect.com",
    "mail.onion.com",
    "mail.spiral.com",
]

params = {
    "struct_file": "../../Drain_result/apache2/mail.cup.com-access.log_structured.csv",
    "template_file": "../../Drain_result/apache2/mail.cup.com-access.log_templates.csv",
    "label_file": f"../../AIT-LDS-v1_1/labels/{label_dir[0]}/apache2/mail.cup.com-access.log",
}

os.makedirs(data_dir, exist_ok=True)


In [2]:
struct_log = pd.read_csv(params['struct_file'])
template_log = pd.read_csv(params['template_file'])
labels = pd.read_csv(params['label_file'])
struct_log['time_label'] = labels['time_label']
struct_log['line_label'] = labels['line_label']

In [3]:
for index, row in struct_log.iterrows():
        struct_log.loc[index, "DateTime"] = parse_datetime(row)

In [4]:
struct_log.sort_values("DateTime", inplace=True)

In [5]:
split_date = pd.to_datetime(struct_log[(struct_log['line_label'] != "0") & (
        struct_log['time_label'] != "0")].DateTime).dt.date.value_counts().sort_index()
split_date.index[0]

datetime.date(2020, 3, 4)

In [6]:
train_set = struct_log[struct_log.DateTime < split_date.index[0]]
test_set = struct_log[struct_log.DateTime >= split_date.index[0]]

/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_3870/2540057205.py:1: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  train_set = struct_log[struct_log.DateTime < split_date.index[0]]
/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_3870/2540057205.py:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  test_set = struct_log[struct_log.DateTime >= split_date.index[0]]


In [7]:
eventId_train = train_set.EventId.unique()
eventId_test = test_set.EventId.unique()

In [8]:
template_log_train = template_log[template_log["EventId"].isin(eventId_train)]
template_log_test = template_log[template_log["EventId"].isin(eventId_test)]
template_log_train["EventTemplateIdent_cleansed"] = template_log_train.EventTemplateIdent.map(text_cleansing)

/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_3870/1766809074.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_log_train["EventTemplateIdent_cleansed"] = template_log_train.EventTemplateIdent.map(text_cleansing)


In [9]:
template_log_train_list = template_log_train["EventTemplateIdent_cleansed"].astype('str').tolist()

In [10]:
tokenizer = RegexpTokenizer(r'[A-Z][a-z]+|\w+')
token_train_list = [tokenizer.tokenize(sen) for sen in template_log_train_list]

template_log_train["EventTemplateIdent_token"] = pd.Series(token_train_list)
train_set["Token"] = train_set.EventId.map(
    lambda id: template_log_train[template_log_train.EventId == id].
    EventTemplateIdent_token.values[0]
)

/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_3870/2881305911.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_log_train["EventTemplateIdent_token"] = pd.Series(token_train_list)
/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_3870/2881305911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set["Token"] = train_set.EventId.map(


In [11]:
trainWord2VecModelType2(token_train_list, "AIT_apache2_word2Vec")

start train word2Vec model. . . . .
finish train word2Vec model . . . . . ^^


In [12]:
with open(os.path.join(data_dir, "train_set.pkl"), "wb") as fw:
    pickle.dump(train_set, fw)
with open(os.path.join(data_dir, "test_set.pkl"), "wb") as fw:
    pickle.dump(test_set, fw)
with open(os.path.join(data_dir, "template_train_set.pkl"), "wb") as fw:
    pickle.dump(template_log_train, fw)
with open(os.path.join(data_dir, "template_test_set.pkl"), "wb") as fw:
    pickle.dump(template_log_test, fw)